In [1]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [2]:
y_actual = tf.constant([[1, 2, 3, 4, 5],[4, 4, 4, 4, 4],[4, 4, 4, 4, 4]], dtype=tf.float32)
y_pred = tf.constant([[2, 4, 6, 8, 10],[4, 4, 4, 4, 4],[4, 4, 4, 4, 4]], dtype=tf.float32)

In [3]:
print(y_actual.shape)
print(y_pred.shape)
print(tf.keras.losses.mean_squared_error(y_actual, y_pred))

(3, 5)
(3, 5)
tf.Tensor([11.  0.  0.], shape=(3,), dtype=float32)


In [4]:
# How to get the last
print(y_actual[:,-1:None])

# How to get the second to last
print(y_actual[:,-2:-1])

tf.Tensor(
[[5.]
 [4.]
 [4.]], shape=(3, 1), dtype=float32)
tf.Tensor(
[[4.]
 [4.]
 [4.]], shape=(3, 1), dtype=float32)


In [5]:
pred_return_ratio = tf.math.divide(tf.math.subtract(y_pred[:, -1:None], y_actual[:, -2:-1]), y_actual[:, -2:-1])
actual_return_ratio = tf.math.divide(tf.math.subtract(y_actual[:, -1:None], y_actual[:, -2:-1]), y_actual[:, -2:-1])

pred_return_ratio = tf.math.divide(tf.math.subtract(y_pred[:, -2:-1], y_actual[:, -3:-2]), y_actual[:, -3:-2])
actual_return_ratio = tf.math.divide(tf.math.subtract(y_actual[:, -2:-1], y_actual[:, -3:-2]), y_actual[:, -3:-2])

In [6]:
print(pred_return_ratio)
print(actual_return_ratio)

tf.Tensor(
[[1.6666666]
 [0.       ]
 [0.       ]], shape=(3, 1), dtype=float32)
tf.Tensor(
[[0.33333334]
 [0.        ]
 [0.        ]], shape=(3, 1), dtype=float32)


In [7]:
all_ones = all_ones = tf.ones([y_actual[:, -1:None].shape[0], 1], dtype=tf.float32)
print(all_ones)

tf.Tensor(
[[1.]
 [1.]
 [1.]], shape=(3, 1), dtype=float32)


In [8]:
# Get the predicted difference matrix
a = tf.matmul(pred_return_ratio, all_ones, transpose_b=True)
# Get the predicted difference matrix
b =tf.matmul(all_ones, pred_return_ratio,transpose_b=True)
tf.subtract(a, b)

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[ 0.       ,  1.6666666,  1.6666666],
       [-1.6666666,  0.       ,  0.       ],
       [-1.6666666,  0.       ,  0.       ]], dtype=float32)>

### Let's convert to multiple timesteps

In [9]:
# Nx1 return ratios need to be NxT

actual_return_ratio_matrix = tf.math.divide(tf.math.subtract(y_actual[:, 1:], y_actual[:, 0:-1]), y_actual[:, 0:-1])
pred_return_ratio_matrix = tf.math.divide(tf.math.subtract(y_pred[:, 1:], y_actual[:, 0:-1]), y_actual[:, 0:-1])
print(y_actual)
print(actual_return_ratio_matrix)
print(y_pred)
print(pred_return_ratio_matrix)

tf.Tensor(
[[1. 2. 3. 4. 5.]
 [4. 4. 4. 4. 4.]
 [4. 4. 4. 4. 4.]], shape=(3, 5), dtype=float32)
tf.Tensor(
[[1.         0.5        0.33333334 0.25      ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]], shape=(3, 4), dtype=float32)
tf.Tensor(
[[ 2.  4.  6.  8. 10.]
 [ 4.  4.  4.  4.  4.]
 [ 4.  4.  4.  4.  4.]], shape=(3, 5), dtype=float32)
tf.Tensor(
[[3.        2.        1.6666666 1.5      ]
 [0.        0.        0.        0.       ]
 [0.        0.        0.        0.       ]], shape=(3, 4), dtype=float32)


In [10]:
# all_ones_matrix = tf.ones(actual_return_ratio_matrix.shape, dtype=tf.float32)
# print(all_ones_matrix)
# tf.einsum('ij,jk->ik', pred_return_ratio_matrix, tf.transpose(all_ones_matrix))

In [11]:
# (tf.repeat(tf.expand_dims(pred_return_ratio_matrix, 1), pred_return_ratio_matrix.shape[-1], axis=1))
t = (tf.repeat(tf.expand_dims(pred_return_ratio_matrix, 2), pred_return_ratio_matrix.shape[0], axis=2))

In [12]:
# Get the predicted difference matrix
a = tf.matmul(pred_return_ratio, all_ones, transpose_b=True)
# Get the predicted difference matrix
b =tf.matmul(all_ones, pred_return_ratio,transpose_b=True)
pred_dif_original = tf.subtract(a, b)

# Get the predicted difference matrix
a = tf.matmul(actual_return_ratio, all_ones, transpose_b=True)
# Get the predicted difference matrix
b = tf.matmul(all_ones, actual_return_ratio,transpose_b=True)
actual_dif_original = tf.subtract(b, a)

In [13]:
p = (tf.repeat(tf.expand_dims(pred_return_ratio_matrix, 2), pred_return_ratio_matrix.shape[0], axis=2))
pred_dif = (p-tf.transpose(p))
pred_dif[:, -2, :]

p = (tf.repeat(tf.expand_dims(actual_return_ratio_matrix, 2), actual_return_ratio_matrix.shape[0], axis=2))
actual_dif = -(p-tf.transpose(p))
actual_dif[:, -2, :]

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[-0.        , -0.33333334, -0.33333334],
       [ 0.33333334, -0.        , -0.        ],
       [ 0.33333334, -0.        , -0.        ]], dtype=float32)>

In [14]:
# Calculate all the different (ri-rj) * (ri-rj)hat sets through time)
s = tf.multiply(pred_dif, actual_dif)
# Remove all negative values
s = tf.nn.relu(s)
# Average across time
s = tf.reduce_mean(s,1)
# Average all values
s = tf.reduce_mean(s)


## How to repeat the NxN matrix across time, so NxTxN

In [79]:
# Recreate Nx1
t_w = tf.constant([[1, 2, 3], [4, 5, 6], [7, 8, 9]], dtype=tf.float32)
# t_w = tf.transpose(t_w)
t_w.shape

TensorShape([3, 3])

In [80]:
# # Orginal output matrix
# all_ones = tf.ones([t_w.shape[0], 1], dtype=tf.float32)
# b = tf.matmul(all_ones, t_w, transpose_b=True)

In [81]:
a = (tf.repeat(tf.expand_dims(t_w, 1), 10, axis=1))
a

<tf.Tensor: shape=(3, 10, 3), dtype=float32, numpy=
array([[[1., 2., 3.],
        [1., 2., 3.],
        [1., 2., 3.],
        [1., 2., 3.],
        [1., 2., 3.],
        [1., 2., 3.],
        [1., 2., 3.],
        [1., 2., 3.],
        [1., 2., 3.],
        [1., 2., 3.]],

       [[4., 5., 6.],
        [4., 5., 6.],
        [4., 5., 6.],
        [4., 5., 6.],
        [4., 5., 6.],
        [4., 5., 6.],
        [4., 5., 6.],
        [4., 5., 6.],
        [4., 5., 6.],
        [4., 5., 6.]],

       [[7., 8., 9.],
        [7., 8., 9.],
        [7., 8., 9.],
        [7., 8., 9.],
        [7., 8., 9.],
        [7., 8., 9.],
        [7., 8., 9.],
        [7., 8., 9.],
        [7., 8., 9.],
        [7., 8., 9.]]], dtype=float32)>

In [82]:
print(a[:, 1, :])

tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]], shape=(3, 3), dtype=float32)


## How does softmax work?

In [85]:
a_n = tf.nn.softmax(a, axis=0)
a_n = tf.nn.softmax(a_n, axis=2)
a_n[:, 1, :]

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[0.33333334, 0.33333334, 0.33333334],
       [0.33333334, 0.33333334, 0.33333334],
       [0.33333334, 0.33333334, 0.33333334]], dtype=float32)>

In [84]:
tf.nn.softmax(t_w, axis=0)

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[0.00235563, 0.00235563, 0.00235563],
       [0.04731416, 0.04731416, 0.04731416],
       [0.95033026, 0.95033026, 0.95033026]], dtype=float32)>